In [ ]:
https://www.kaggle.com/code/taronzakaryan/predicting-stock-price-using-lstm-model-pytorch

In [14]:
import torch
import torch.nn as nn
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [40]:
df = pd.read_csv('data/EURUSD/M15.csv', index_col=False)
df = df.drop(columns=['time'])
df = df[:8700]

In [41]:
df.head()

,open,high,low,close,tick_volume,volume_adi,volume_obv,volume_cmf,volume_fi,volume_em,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,1.14145,1.14148,1.14095,1.14127,408,84.679245,408,0.207547,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.141270,0.792751,0.000000,0.000000
1,1.14128,1.14134,1.14095,1.14129,295,304.038220,703,0.432487,0.005900,-0.009254,...,0.000140,0.000028,0.000112,-2.255650,-0.451130,-1.804520,1.141279,0.001752,0.001752,0.001752
2,1.14130,1.14188,1.14103,1.14184,415,679.979396,1118,0.608211,0.037664,0.063494,...,0.004092,0.000841,0.003251,-1.597260,-0.680356,-0.916904,1.141523,0.048191,0.048179,0.049944
3,1.14184,1.14209,1.14103,1.14110,610,150.545434,508,0.087121,-0.032202,0.018246,...,0.001970,0.001067,0.000903,2.708446,-0.002596,2.711042,1.141342,-0.064808,-0.064829,-0.014896
4,1.14110,1.14137,1.14093,1.14118,388,203.454525,896,0.096151,-0.023167,-0.046495,...,0.000843,0.001022,-0.000179,1.758913,0.349706,1.409207,1.141273,0.007011,0.007011,-0.007886


In [42]:
#df = pd.read_csv('data/EURUSD/M15.csv', index_col=False)
target = pd.DataFrame(df.close).pct_change()
target = target > 0
target = target.replace(True, 1)
target = target.replace(False, 0)
#removing the first line because we use pct_change 
df = df[1:]
target = target[1:]

In [43]:
window_size = 20
n = np.array(df)
x = np.array([n[i-window_size:i] for i in range(window_size, len(df))], dtype=object)
#df = torch.from_numpy(x.astype(np.float32))
df = x.astype(np.float32)
target = target[window_size:]

In [44]:
train_size = int(len(df)*0.8)
x_train = df[:train_size]
y_train = np.array(target[:train_size]).astype(np.float32)
x_test = df[train_size:]
y_test =  np.array(target[train_size:]).astype(np.float32)

In [45]:
print(f'Shape of x_train is {x_train.shape}')
print(f'Shape of x_test is {x_test.shape}')
print(f'Shape of y_train is {y_train.shape}')
print(f'Shape of y_test is {y_test.shape}')

Shape of x_train is (6943, 20, 91)
Shape of x_test is (1736, 20, 91)
Shape of y_train is (6943, 1)
Shape of y_test is (1736, 1)


In [46]:
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)
y_train.size(),x_train.size()

(torch.Size([6943, 1]), torch.Size([6943, 20, 91]))

In [47]:
sequence_length = 28
input_size = 91
hidden_size = 128
num_layers = 16
num_classes = 2
batch_size = 1000
num_epochs = 5
learning_rate = 0.0001

In [48]:
class RNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, hidden = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        #Reshaping the outputs such that it can be fit into the fully connected layer
        out = self.fc(out[:, -1, :])
        return out
        
        
model = RNN(input_size, hidden_size, num_layers, num_classes)
print(model)
     

RNN(
  (lstm): LSTM(91, 128, num_layers=16, batch_first=True)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)


In [49]:
loss_func = nn.CrossEntropyLoss()
from torch import optim
optimizer = optim.Adam(model.parameters(), lr = 0.001) 
     

In [ ]:
def train(num_epochs, model):
        
    # Train the model
    total_step = len(x_train)
    correct_preds = 0
    for epoch in range(num_epochs):
        print("-"*50,f' Epoch {epoch+1} ',"-"*50)
        for i in range(total_step):
            
            # Forward pass
            outputs = model((x_train[i].reshape(-1,x_train[i].size(0) , x_train[i].size(1))))
            outputs = outputs.float()
            if y_train[i].item() == 1:
                target = torch.tensor([1]).long()
            elif y_train[i].item() == 0:
                target = torch.tensor([0]).long()
            loss = loss_func(outputs, target)
            correct_preds += target.eq(outputs.argmax()).sum().item()
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (i+1) % batch_size == 0 or (i+1) == total_step:
                denom = batch_size if i < total_step-1 else total_step % batch_size
                accuracy = correct_preds / batch_size
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.2f} | Accuracy: {:.2f}' 
                       .format(epoch + 1, num_epochs, i+1, total_step, loss.item(), accuracy))
                correct_preds = 0
                


train(num_epochs, model)

--------------------------------------------------  Epoch 1  --------------------------------------------------
Epoch [1/5], Step [1000/6943], Loss: 0.74 | Accuracy: 0.51
Epoch [1/5], Step [2000/6943], Loss: 0.71 | Accuracy: 0.50
Epoch [1/5], Step [3000/6943], Loss: 0.68 | Accuracy: 0.49
Epoch [1/5], Step [4000/6943], Loss: 0.71 | Accuracy: 0.51
Epoch [1/5], Step [5000/6943], Loss: 0.70 | Accuracy: 0.48
Epoch [1/5], Step [6000/6943], Loss: 0.71 | Accuracy: 0.49
Epoch [1/5], Step [6943/6943], Loss: 0.72 | Accuracy: 0.48
--------------------------------------------------  Epoch 2  --------------------------------------------------
Epoch [2/5], Step [1000/6943], Loss: 0.73 | Accuracy: 0.53
Epoch [2/5], Step [2000/6943], Loss: 0.70 | Accuracy: 0.49
Epoch [2/5], Step [3000/6943], Loss: 0.68 | Accuracy: 0.50
Epoch [2/5], Step [4000/6943], Loss: 0.71 | Accuracy: 0.50
Epoch [2/5], Step [5000/6943], Loss: 0.70 | Accuracy: 0.48
Epoch [2/5], Step [6000/6943], Loss: 0.71 | Accuracy: 0.49
Epoch [2/